[Last time](https://terrenceho.github.io/functional/2017/10/21/From-Object-Oriented-To-Functional/), we went over using functions to define behavior.  Now for a more practical example, we'll discuss a pattern to define options of a class or API, one that can grow without bloat and is easly discoverable.

A simple way to define configurations is just to set a bunch of configuration parameters.  

TODO:
Define function schema 1 for Python
```
def FunctionName(parameters):
    def ConfigOperation(f):
        // modify f...
    return ConfigOperation
```
Define function schema 2 for Python, for optional types and return prev value
```
def FunctionName(parameters):
    def ConfigOperation(f):
        // retrieve and store previous value
        // modify f...
        // return previous value
    return ConfigOperation
```

Define function schema 3 for Python, for optional types, and allows you to return the last args previous value
```
def FunctionName(parameters):
    def ConfigOperation(f):
        // retrieve and store previous value
        // modify f...
        // return FunctionName(previous value)
    return ConfigOperation
```

In [1]:
class Foo(object):
    # Calls Options function and passes in function args, to allow reuseability
    def __init__(self, *args):
        self.verbosity = 0
        # allocate variables
        self.Options(*args)
            
    def Options(self, *args):
        for func in args:
            func(self)

In [2]:
def Verbosity(v):
    def ConfigOperation(f):
        f.verbosity = v
    return ConfigOperation

f = Foo(Verbosity(5))
print(f.verbosity)

5


In [3]:
def Verbosity(v):
    def ConfigOperation(f):
        prev = f.verbosity
        f.verbosity = v
        return prev
    return ConfigOperation

f = Foo(Verbosity(5))
print(f.verbosity)

5


In [4]:
def Verbosity(v):
    def ConfigOperation(f):
        prev = f.verbosity
        f.verbosity = v
        return Verbosity(prev)
    return ConfigOperation

f = Foo(Verbosity(5))
print(f.verbosity)

5


Let's say we want to make a server module.  It starts out pretty simple.

In [ ]:
import socket
    
class Server(object):
    def __init__(self, HOST, PORT):
        self.HOST = HOST
        self.PORT = PORT
        
        
    def listen(self):
        self.listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.listen_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.listen_socket.bind((self.HOST, self.PORT))
        self.listen_socket.listen(1)
        print('Serving HTTP on port %s ...' % self.PORT)
        while True:
            client_connection, client_address = self.listen_socket.accept()
            request = client_connection.recv(1024)
            print(request.decode('utf-8'))
            http_response = """\
HTTP/1.1 200 OK

Hello, World!
"""
            client_connection.sendall(http_response.encode('utf-8'))
            client_connection.close()
            
newServer = Server("localhost", 8080) # Listen on localhost, port 8080
newServer.listen()

Serving HTTP on port 8080 ...
GET / HTTP/1.1
Host: localhost:8080
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Upgrade-Insecure-Requests: 1
Cookie: _ga=GA1.1.1281775909.1508703278
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/603.2.4 (KHTML, like Gecko) Version/10.1.1 Safari/603.2.4
Accept-Language: en-us
Accept-Encoding: gzip, deflate
Connection: keep-alive




The server is simple.  It listens on the specified address and port for a connection, andThere's a problem... As soon as you release this module, it blows up in popularity, and the once simple configuration becomes long and ugly.